<a href="https://colab.research.google.com/github/Ravi-kjain84/Articles/blob/main/251%20-%20Determinants%20of%20non-performing%20loans%3A%20Evidence%20from%20Euro-area%20countries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Load the data from the Excel file
file_path = '/content/NPL_research_template.xlsx'  # Replace with your file path
data = pd.read_excel(file_path, sheet_name="main")
data.fillna(0, inplace=True)

# Lagged variables
data['NPL_lagged'] = data['NPL'].shift(1)
data['ROA_lagged'] = data['ROA'].shift(1)
data['ROE_lagged'] = data['ROE'].shift(1)
data['LTD_lagged'] = data['LTD'].shift(1)
data['UNEMP_lagged'] = data['UNEMP'].shift(1)
data['TAXINC_lagged'] = data['TAXINC'].shift(1)
data['FISCAL_lagged'] = data['FISCAL'].shift(1)
data['DEBT_lagged'] = data['DEBT'].shift(1)
data['GROWTH_lagged'] = data['GROWTH'].shift(1)
data['INFLRAT_lagged'] = data['INFLRAT'].shift(1)
data['OUTPUT_GAP_lagged'] = data['OUTPUT_GAP'].shift(1)

# Dropping NA values
data.dropna(inplace=True)

# Dependent variable
Y = data['NPL']

# Independent variables
X = data[['ROA', 'ROE', 'LTD', 'UNEMP', 'DEBT', 'GROWTH', 'INFLRAT',
          'ROA_lagged', 'ROE_lagged', 'LTD_lagged', 'UNEMP_lagged',
          'DEBT_lagged', 'GROWTH_lagged', 'INFLRAT_lagged']]

# Adding a constant to the independent variables
X = sm.add_constant(X)

# Check for multicollinearity using VIF
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print(vif_data)

# Remove variables with high VIF (> 5)
high_vif_features = vif_data[vif_data['VIF'] > 5]['Feature']
X = X.drop(columns=high_vif_features)

# Instrumental variables (for GMM estimation)
instruments = X.copy()
instruments['NPL_lagged'] = data['NPL_lagged']

# Performing the GMM estimation
model = IV2SLS(Y, X, instruments).fit()

# Printing the summary of the results
print(model.summary())


           Feature        VIF
0            const   9.686714
1              ROA   4.526241
2              ROE   4.087759
3              LTD   1.005257
4            UNEMP  38.275020
5             DEBT  10.578660
6           GROWTH   5.753046
7          INFLRAT   2.853993
8       ROA_lagged   4.533048
9       ROE_lagged   4.107344
10      LTD_lagged   1.061986
11    UNEMP_lagged  38.182136
12     DEBT_lagged  10.686944
13   GROWTH_lagged   5.694981
14  INFLRAT_lagged   2.865687
                          IV2SLS Regression Results                           
Dep. Variable:                    NPL   R-squared:                       0.064
Model:                         IV2SLS   Adj. R-squared:                  0.062
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Sat, 18 May 2024                                         
Time:                        16:05:20         